In [1]:
%run src/init.py basic

<span style='color:green'>**Configuration Ready (cse-in)**</span>

In [2]:
CREATE (                                                     # CREATE request
    
    # Create the AE resource under the CSEBase
    to                      = cseBaseName,  
    
    # Request Parameters
    originator              = 'Cmyself',                     # Assign an originator ID, must start with 'C'
    requestIdentifier       = '123',                         # Unique request identifier
    releaseVersionIndicator = '3',                           # Release version indicator
    resourceType            = Type.AE,                       # Type of the resource: AE
    
    # Request Body
    
    primitiveContent = 
        {   
            'm2m:ApplicationEntity': {
                'resourceName':             'TempSensor',   # Name of the resource 
                'App-ID':                   'NTempSensor',   # Application ID, must start with 'N'
                'requestReachability':      True,            # AE can receive requests
                'supportedReleaseVersions': [ '3' ]          # Supports oneM2M release 3
            }
        }
)


HTTP Header,Request Attribute,Value
X-M2M-Origin,From,Cmyself
X-M2M-RI,Request Identifier,123
X-M2M-RVI,Release Version Indicator,3
Content-Type,,application/json;ty=2
Accept,,application/json


HTTP Header,Request Attribute,Value
X-M2M-RSC,Response Status Code,2001
X-M2M-RI,Request Identifier,123
X-M2M-RVI,Release Version Indicator,3
X-M2M-OT,Originating Timestamp,"20240104T195644,353979"
Content-Type,,application/json


---

In [3]:
CREATE (                                      # CREATE request
    
    # Create the container resource under the AE
    to                      = cseBaseName + '/TempSensor', 

    # Request Parameters
    originator              = 'Cmyself',      # Set the originator
    requestIdentifier       = '123',          # Unique request identifier
    releaseVersionIndicator = '3',            # Release version indicator
    resourceType            = Type.Container, # Type of the resource: container
    
    # Request Body
    primitiveContent = 
        {
            'm2m:Container': {
                'resourceName': 'Container'   # Set the resource name
            }
        },
)
    

HTTP Header,Request Attribute,Value
X-M2M-Origin,From,Cmyself
X-M2M-RI,Request Identifier,123
X-M2M-RVI,Release Version Indicator,3
Content-Type,,application/json;ty=3
Accept,,application/json


HTTP Header,Request Attribute,Value
X-M2M-RSC,Response Status Code,2001
X-M2M-RI,Request Identifier,123
X-M2M-RVI,Release Version Indicator,3
X-M2M-OT,Originating Timestamp,"20240104T195652,668676"
Content-Type,,application/json


---

In [ ]:
import random
import time
import threading

file_path = "C:/Users/user/OneDrive/Bureau/QoS-optimization-in-IoT-systems-main/telemetry.txt"
def generate_simulated_data(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    random_values = random.sample(lines, 100)  # Select 100 random values from the file
    return [float(line.strip()) for line in random_values]


def send_data_to_onem2m_server(data):
    start_time = time.time()
    CREATE (                                            # CREATE request

        # Create the content instance resource under the container
        to                      = cseBaseName + '/TempSensor/Container', 

        # Request Parameters
        originator              = 'Cmyself',            # Set the originator
        requestIdentifier       = '123',                # Unique request identifier
        releaseVersionIndicator = '3',                  # Release version indicator
        resourceType            = Type.ContentInstance, # Type of the resource: contentInstance
        
        # Request Body
        primitiveContent = 
            {
            "m2m:cin": {
                "cnf": "text/plain:0",
                "con": str(data),
            }
               } )
    end_time = time.time()
    rtt = (end_time - start_time)
    print("the RTT is:",rtt)
    return rtt

lock = threading.Lock()

def simulate_device_behavior():
    global data_sent
    data_sent = 0

    while True:
        start_iteration = time.time()
        simulated_data = generate_simulated_data(file_path)
        rtt = send_data_to_onem2m_server(simulated_data)
        rtt -= 4.0  # Subtract any processing time

        # Count the number of data sent
        with lock:
            if data_sent >= 100:
                simulated_data = []  # No data sent if previous thread sent the maximum
            elif data_sent + len(simulated_data) > 100:
                remaining_space = 100 - data_sent
                simulated_data = simulated_data[:remaining_space]
                data_sent = 100
            else:
                data_sent += len(simulated_data)

        with open('./rtt_values.txt', 'a') as file:
            file.write(str(rtt) + '\n')

        # Control loop to maintain sending 100 data points per second
        end_iteration = time.time()
        iteration_time = end_iteration - start_iteration

        if iteration_time < 1 / 100:
            time.sleep(1 / 100 - iteration_time)
        else:
            # Write total sent data each second to a file
            with open('./total_data_sent_per_second.txt', 'a') as file:
                file.write(str(data_sent) + "\n")
        data_sent = 0  # Reset data_sent count for the next second

  
# Create three threads to simulate three traffic sources
for _ in range(3):
    threading.Thread(target=simulate_device_behavior).start()

---

the RTT is: 4.163532018661499


---

---

the RTT is: 4.185937881469727
the RTT is: 4.185937881469727


...

HTTP Header,Request Attribute,Value
X-M2M-RSC,Response Status Code,2001
X-M2M-RI,Request Identifier,123
X-M2M-RVI,Release Version Indicator,3
X-M2M-OT,Originating Timestamp,"20240103T010451,050396"
Content-Type,,application/json


HTTP Header,Request Attribute,Value
X-M2M-Origin,From,Cmyself
X-M2M-RI,Request Identifier,123
X-M2M-RVI,Release Version Indicator,3
Content-Type,,application/json;ty=4
Accept,,application/json


HTTP Header,Request Attribute,Value
X-M2M-RSC,Response Status Code,2001
X-M2M-RI,Request Identifier,123
X-M2M-RVI,Release Version Indicator,3
X-M2M-OT,Originating Timestamp,"20240103T010451,095398"
Content-Type,,application/json


HTTP Header,Request Attribute,Value
X-M2M-Origin,From,Cmyself
X-M2M-RI,Request Identifier,123
X-M2M-RVI,Release Version Indicator,3
Content-Type,,application/json;ty=4
Accept,,application/json


HTTP Header,Request Attribute,Value
X-M2M-RSC,Response Status Code,2001
X-M2M-RI,Request Identifier,123
X-M2M-RVI,Release Version Indicator,3
X-M2M-OT,Originating Timestamp,"20240103T010451,160395"
Content-Type,,application/json


In [ ]:
"""
import random
import time
import csv 

file_path = "C:/Users/user/OneDrive/Bureau/QoS-optimization-in-IoT-systems-main/temperature.txt"  # Replace with your desired file path


def generate_simulated_data():
    # Simulate temperature readings between 20 and 30 degrees Celsius
    return round(random.uniform(20, 30), 2)

def send_data_to_onem2m_server(data):
    start_time = time.time()
    CREATE (                                            # CREATE request

        # Create the content instance resource under the container
        to                      = cseBaseName + '/TempSensor/Container', 

        # Request Parameters
        originator              = 'Cmyself',            # Set the originator
        requestIdentifier       = '123',                # Unique request identifier
        releaseVersionIndicator = '3',                  # Release version indicator
        resourceType            = Type.ContentInstance, # Type of the resource: contentInstance
        
        # Request Body
        primitiveContent = 
            {
            "m2m:cin": {
                "cnf": "text/plain:0",
                "con": str(data),
            }
               } )
    
    end_time = time.time()
    rtt = (end_time - start_time)
    return rtt

def simulate_device_behavior():
    L=[]
    while True : 
        simulated_data = generate_simulated_data()
        rtt = send_data_to_onem2m_server(simulated_data)
        L.append(rtt)
        # Write the RTT values to a text file
        with open('./rtt_values.txt', 'a') as file:
            file.write(str(rtt) + '\n')

         # Adjust the delay as needed
# Start simulating device behavior
simulate_device_behavior()"""

'\nimport random\nimport time\nimport csv \n\nfile_path = "C:/Users/user/OneDrive/Bureau/QoS-optimization-in-IoT-systems-main/temperature.txt"  # Replace with your desired file path\n\n\ndef generate_simulated_data():\n    # Simulate temperature readings between 20 and 30 degrees Celsius\n    return round(random.uniform(20, 30), 2)\n\ndef send_data_to_onem2m_server(data):\n    start_time = time.time()\n    CREATE (                                            # CREATE request\n\n        # Create the content instance resource under the container\n        to                      = cseBaseName + \'/TempSensor/Container\', \n\n        # Request Parameters\n        originator              = \'Cmyself\',            # Set the originator\n        requestIdentifier       = \'123\',                # Unique request identifier\n        releaseVersionIndicator = \'3\',                  # Release version indicator\n        resourceType            = Type.ContentInstance, # Type of the resource: content

HTTP Header,Request Attribute,Value
X-M2M-Origin,From,Cmyself
X-M2M-RI,Request Identifier,123
X-M2M-RVI,Release Version Indicator,3
Content-Type,,application/json;ty=4
Accept,,application/json


HTTP Header,Request Attribute,Value
X-M2M-Origin,From,Cmyself
X-M2M-RI,Request Identifier,123
X-M2M-RVI,Release Version Indicator,3
Content-Type,,application/json;ty=4
Accept,,application/json


HTTP Header,Request Attribute,Value
X-M2M-Origin,From,Cmyself
X-M2M-RI,Request Identifier,123
X-M2M-RVI,Release Version Indicator,3
Content-Type,,application/json;ty=4
Accept,,application/json


HTTP Header,Request Attribute,Value
X-M2M-RSC,Response Status Code,2001
X-M2M-RI,Request Identifier,123
X-M2M-RVI,Release Version Indicator,3
X-M2M-OT,Originating Timestamp,"20240103T010356,963081"
Content-Type,,application/json


...

HTTP Header,Request Attribute,Value
X-M2M-RSC,Response Status Code,2001
X-M2M-RI,Request Identifier,123
X-M2M-RVI,Release Version Indicator,3
X-M2M-OT,Originating Timestamp,"20240103T010434,359546"
Content-Type,,application/json


HTTP Header,Request Attribute,Value
X-M2M-Origin,From,Cmyself
X-M2M-RI,Request Identifier,123
X-M2M-RVI,Release Version Indicator,3
Content-Type,,application/json;ty=4
Accept,,application/json


HTTP Header,Request Attribute,Value
X-M2M-RSC,Response Status Code,2001
X-M2M-RI,Request Identifier,123
X-M2M-RVI,Release Version Indicator,3
X-M2M-OT,Originating Timestamp,"20240103T010434,484451"
Content-Type,,application/json


HTTP Header,Request Attribute,Value
X-M2M-Origin,From,Cmyself
X-M2M-RI,Request Identifier,123
X-M2M-RVI,Release Version Indicator,3
Content-Type,,application/json;ty=4
Accept,,application/json


HTTP Header,Request Attribute,Value
X-M2M-RSC,Response Status Code,2001
X-M2M-RI,Request Identifier,123
X-M2M-RVI,Release Version Indicator,3
X-M2M-OT,Originating Timestamp,"20240103T010434,515819"
Content-Type,,application/json
